In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

GFG Jobathon: 32

Q1: Lead Engineer

In [5]:
def leadOptions(self, N : int, E : List[int]) -> int:
    maxE: tuple[int, int] = -1, 0
    for exp in E:
        if exp == maxE[0]:
            maxE = maxE[0], maxE[1] + 1
        if exp > maxE[0]:
            maxE = exp, 0

    return maxE[1]

Codeforces: 20/C (Dijkstra)

In [6]:
def dijkstra(src: int, adjl: dict[int, list[tuple[int, int]]]) -> dict[int, tuple[float, int]]:
    heap: list[tuple[int, int, int]] = [(1, src, 1)]
    shortest: dict[int, tuple[float, int]] = dict()
    while heap:
        curr_dist, curr, prev = heapq.heappop(heap)
        if curr not in shortest or curr_dist < shortest[curr][0]:
            shortest[curr] = (curr_dist, prev)
            for next_, next_dist in adjl.get(curr, []):
                heapq.heappush(heap, (curr_dist + next_dist, next_, curr))

    return shortest

def twentyC(V: int, E: int, edges: list[tuple[int, int, int]]) -> str:
    adjl: dict[int, list[tuple[int, int]]] = dict()
    for n1, n2, d in edges:
        n1_neighbours, n2_neighbours = adjl.get(n1, []), adjl.get(n2, [])
        n1_neighbours.append((n2, d))
        n2_neighbours.append((n1, d))
        adjl[n1], adjl[n2] = n1_neighbours, n2_neighbours

    paths = dijkstra(1, adjl)
    if V in paths:
        result: list[int] = [V]
        curr = V
        while curr != 1:
            dist, prev = paths[curr]
            result.append(prev)
            curr = prev

        result.reverse()
        return ' '.join(map(str, result))
    else:
        return '-1'

# Testing the solution
assert twentyC(5, 5, [(1, 2, 2), (2, 5, 5), (2, 3, 4), (1, 4, 1), (4, 3, 3), (3, 5, 1)]) == '1 4 3 5'

LC Medium: Course Schedule 2

In [7]:
# https://leetcode.com/problems/course-schedule-ii/submissions/1239496066
def findOrder(numCourses: int, prerequisites: list[list[int]]) -> list[int]:
    adjl: dict[int, list[int]] = dict()
    for c1, c2 in prerequisites:
        # To take up C1, C2 must be done already
        c1_neighbours: list[int] = adjl.get(c1, [])
        c1_neighbours.append(c2)
        adjl[c1] = c1_neighbours

    paths: dict[int, list[int]] = dict()
    def getOrdering(curr: int, visited: set[int] = set()) -> list[int]:
        if curr in paths:
            return paths[curr]
        elif curr in visited:
            return []
        else:
            result: list[int] = [curr]
            visited.add(curr)
            for next_ in adjl.get(curr, []):
                deps: list[int] = getOrdering(next_, visited)
                if not deps:
                    return []
                else:
                    result.extend(deps)

            visited.remove(curr)
            paths[curr] = result
            return result

    # Courses are ordered from 0 to numCourses - 1
    result: list[int] = []
    completed: set[int] = set()
    for course in range(numCourses):
        deps = getOrdering(course)
        deps.reverse()
        if not deps:
            return []
        else:
            for dep in deps:
                if dep not in completed:
                    completed.add(dep)
                    result.append(dep)

    return result

# Testing the solution
assert findOrder(4, [[1,0], [2,0], [3,1], [3,2]]) in ([0,1,2,3], [0,2,1,3])
assert findOrder(5, [[1,0], [2,0], [3,4], [4,3]]) == []

LC Medium: Course Schedule 1 (DFS)

In [8]:
# https://leetcode.com/problems/course-schedule/submissions/1240858033/
def canFinish(numCourses: int, prerequisites: list[list[int]]) -> bool:
    # Convert pre-req to adj list
    adj: dict[int, list[int]] = dict()
    for n1, n2 in prerequisites:
        n1_neighbours = adj.get(n1, [])
        n1_neighbours.append(n2)
        adj[n1] = n1_neighbours

    # DFS Algorithm to check for cycles
    def hasCycle(curr: int, paths: set[int] = set()) -> bool:
        if curr in paths:
            return True
        elif curr in visited:
            return False
        else:
            visited.add(curr)
            paths.add(curr)
            for next_ in adj.get(curr, []):
                if hasCycle(next_, paths):
                    return True
            paths.remove(curr)
            return False

    visited: set[int] = set()
    for curr in range(numCourses):
        if hasCycle(curr):
            return False
    else:
        return True

# Testing the solution
assert canFinish(2, [[1,0]]) == True
assert canFinish(2, [[1,0],[0,1]]) == False

Shortest path in undirected graph with unit weights

In [9]:
def shortestPathUG(n: int, edges: list[tuple[int, int]], src: int) -> list[int]:
    # Create an adj list
    adj: dict[int, list[int]] = dict()
    for n1, n2 in edges:
        n1_neighbours, n2_neighbours = adj.get(n1, []), adj.get(n2, [])
        n1_neighbours.append(n2)
        n2_neighbours.append(n1)
        adj[n1], adj[n2] = n1_neighbours, n2_neighbours

    shortest: list[int] = [-1 for _ in range(n)]
    heap: list[tuple[int, int]] = [(0, src)]
    while heap:
        curr_dist, curr = heapq.heappop(heap)
        shortest[curr] = curr_dist if shortest[curr] == -1 or shortest[curr] > curr_dist else shortest[curr]
        for next_ in adj.get(curr, []):
            if shortest[next_] == -1 or curr_dist + 1 < shortest[next_]:
                heapq.heappush(heap, (curr_dist + 1, next_))

    return shortest

# Testing the solution
assert shortestPathUG(5, [(0, 1), (1, 4), (2, 3), (2, 4), (3, 4)], 1) == [1, 0, 2, 2, 1]

Shortest path in DAG

In [10]:
def shortestPathInDAG(N: int, M: int, edges: list[list[int]]) -> list[int]:
    # Create an adj list
    adj: dict[int, list[tuple[int, int]]] = dict()
    for n1, n2, w in edges:
        n1_neighbours = adj.get(n1, [])
        n1_neighbours.append((n2, w))
        adj[n1] = n1_neighbours

    # Dijkstra's algo!
    shortest: list[int] = [-1 for _ in range(N)]
    heap: list[tuple[int, int]] = [(0, 0)]
    while heap:
        curr_dist, curr = heapq.heappop(heap)
        if shortest[curr] == -1 or curr_dist < shortest[curr]:
            shortest[curr] = curr_dist
            for next_, next_dist in adj.get(curr, []):
                heapq.heappush(heap, (next_dist + curr_dist, next_))

    return shortest

# Testing the solution
assert shortestPathInDAG(3, 3, [[0,1,2], [1,2,3], [0,2,6]]) == [0, 2, 5]
assert shortestPathInDAG(3, 3, [[2,0,4],[0,1,3],[2,1,2]]) == [0, 3, -1]

LC Hard: Word Ladder

In [ ]:
# https://leetcode.com/problems/word-ladder/submissions/1241826378
def ladderLengthBrute(beginWord: str, endWord: str, wordList: list[str]) -> int:
    def pathExists(w1: str, w2: str) -> bool:
        N, i = len(w1), 0
        dist = 0
        while i < N:
            if w1[i] != w2[i]:
                dist += 1
            if dist > 1:
                break
            i += 1

        return dist == 1

    def BFS(root: str):
        queue: collections.deque = collections.deque([root])
        distances[root] = 0
        while queue:
            curr = queue.popleft()
            for next_ in adj.get(curr, []):
                if next_ not in distances or distances[next_] > distances[curr] + 1:
                    distances[next_] = distances[curr] + 1
                    queue.append(next_)
                    if next_ == endWord:
                        return

    # Create an adjacency list
    adj: dict[str, list[str]] = dict()
    for w1, w2 in itertools.combinations([beginWord] + wordList, r=2):
        if pathExists(w1, w2):
            w1_neighbours, w2_neighbours = adj.get(w1, []), adj.get(w2, [])
            w1_neighbours.append(w2)
            w2_neighbours.append(w1)
            adj[w1], adj[w2] = w1_neighbours, w2_neighbours

    distances: dict[str, int] = dict()
    BFS(beginWord)
    return distances[endWord] + 1 if endWord in distances else 0

# Testing the solution
assert ladderLengthBrute("hit", "cog", ["hot","dot","dog","lot","log","cog"]) == 5
assert ladderLengthBrute("hit", "cog", ["hot","dot","dog","lot","log"]) == 0